# Exploring the Julia Set in Julia

[Julia sets](https://en.wikipedia.org/wiki/Julia_set) are beautiful mathematical objects, created by iterating functions over the complex numbers.

We won't go into the mathematics, but will see how to play with them in Julia.

The following Julia code takes as input two complex numbers, and calculates the "escape time" that we will use to plot Julia sets:

In [ ]:
const maxiter = 100

doc"""
Compute the *escape time* of the Julia set at $(z, c)$.  
"""
function julia(z, c)  # keyword argument
    
    for n in 1:maxiter
        abs2(z) > 4 && return n     # "short-circuiting" and = if-then
        z = z*z + c
    end
    
    return maxiter
end

In [ ]:
?Main.julia

(Note that we can use Markdown syntax in the docstring, and LaTeX when using `doc"""..."""` instead of """...""".)

In [ ]:
julia(-0.06 + 0.665im, -0.06 + 0.665im)

In [ ]:
@code_native julia(-0.02 + 0.7im, -0.06 + 0.665im)

We can calculate a Julia set with fixed value of `c` using an array comprehension:

In [ ]:
[julia(r + i*im, -0.06 + 0.665im) for r in -1:0.002:1, i in -0.5:0.002:0.5]

## Adding some colour

In [ ]:
# Install required packages:
# Pkg.add("Images")
# Pkg.add("ImageMagick")
# Pkg.add("QuartzImageIO")

In [ ]:
using Colors, Images, ImageMagick

In [ ]:
const cmap = colormap("RdBu", 100)

In [ ]:
?colormap

In [ ]:
# const cmap = convert(Vector{RGB{U8}}, colormap("RdBu", 100))

In [ ]:
cmap[1]

In [ ]:
typeof(cmap)

In [ ]:
typeof(cmap[1])

In [ ]:
?RGB

In [ ]:
julia(-1.5+0.05im, -0.07+0.66im)

In [ ]:
domain = -1.5 : 0.005 : 1.5

data = [julia(r + i*im, -0.07 + 0.66im) for r in domain, i in domain]

In [ ]:
data[100:200, 100:200]

In [ ]:
pixels = [ cmap[pixel_value] for pixel_value in data ]

In [ ]:
typeof(ans)

In [ ]:
img = Image(pixels)

In [ ]:
typeof(img)

In [ ]:
save("Julia_pixels", pixels)

In [ ]:
save("Julia_set.png", img)

The following is more efficient, at least in memory, since it does not create the intermediate `data` array:

In [ ]:
domain = -1.5:0.005:1.5

pixels = [ cmap[ julia(r + i*im, r + i*im) ] for r in domain-0.5, i in domain]

Image(pixels)

In [ ]:
using Interact

In [ ]:
@manipulate for r′ = -0.1:0.01:0.1, i′ = 0.65:0.01:0.85
    Image(cmap[[julia(r + i*im, r′ + i′*im) for r=-1.5:.005:1.5, i=-1:.005:1]])
end

In [ ]:
# let R = -1.5:.005:1.5, I = -1:.005:1
#     data = Array{RGB{U8}}(length(R), length(I))
#     img = Image(data)
    
#     @manipulate for r′ = -0.1:0.01:0.1, i′ = 0.65:0.01:0.85
#         for (j,r) in enumerate(R), (k,i) in enumerate(I)
#             data[j,k] = cmap[julia(r + i*im, r′ + i′*im)]
#         end
        
#         img
#     end
# end

## Let's animate!

In [ ]:
@manipulate for r′ = -0.1:0.01:0.1, i′ = map(_->0.65+0.5*sin(time())*0.25, fps(10))
    Image([cmap[julia(r + i*im, r′ + i′*im)] for r=-1.5:.005:1.5, i=-1:.005:1])
    end